In [ ]:
#Dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import psycopg2

from config import protocol, username, password, host, port, database_name


In [ ]:
#Extracting Files (Kdrama)
kdrama_file = "./Data/imdb.csv"
kdrama_df = pd.read_csv(kdrama_file)
kdrama_df.head()

In [ ]:
#Cleaning Files (kdrama)
cleaned_kdrama_df = kdrama_df[["kdrama_name", "imdb_rating"]].copy()
cleaned_kdrama_df.sort_values(by=['imdb_rating'], inplace=True, ascending=False)
cleaned_kdrama_df.head()

In [ ]:
##Extracting Files (Genres)
kdrama_genre_file = "./Data/genres.csv"
kdrama_genre_file_df = pd.read_csv(kdrama_genre_file)
kdrama_genre_file_df.head()

In [ ]:
#Merging dataframes 
kdrama_joined= pd.merge(cleaned_kdrama_df, kdrama_genre_file_df, on='kdrama_name', how='inner')
kdrama_joined.head()

In [ ]:
#Capitalizing Genre
kdrama_joined["genre"] = kdrama_joined["genre"].str.upper()
kdrama_joined.head()

In [ ]:
#Appending Genres into One Row
kdrama_joined_genre_df = kdrama_joined.groupby(['kdrama_name'])['genre'].apply(lambda x : np.unique(list(x))).reset_index()


In [ ]:
kdrama_joined_genre_df['genre'] = kdrama_joined_genre_df['genre'].apply(lambda x: x.tolist())

In [ ]:
#Recreating IMDB Rating Dataframe 
kdrama_joined = kdrama_joined[["kdrama_name", "imdb_rating"]]
kdrama_joined.head()


In [ ]:
#Combining Rating and Genre Dataframes
ratings_genre_combined = pd.merge(kdrama_joined, kdrama_joined_genre_df, on='kdrama_name', how='inner')
final_df = ratings_genre_combined.drop_duplicates(subset="kdrama_name")
final_df.head()

In [ ]:
#Removing all IMDB Rating < 8 
final_df = final_df[final_df['imdb_rating'] >= 8] 
final_df.head()

In [ ]:
#Titlecasing Kdrama Name
final_df["kdrama_name"] = final_df["kdrama_name"].str.title()
final_df.head()

In [ ]:
#Resetting Dataframe Index
final_df.reset_index(drop=True, inplace=True)
final_df.head()

In [ ]:
connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(connection_string)

In [ ]:
engine.table_names()

In [ ]:
final_df.to_sql(name='kdrama', con=engine, if_exists='append', index=False)